In [2]:
import pandas as pd
# Excel 파일에 sheet가 여러 개이면 sheet_name = "sheet 이름"을
# read_excel() 함수에 지정하면 되고, sheet_name 매개 변수에
# 아무 값도 주지 않으면 첫 번째 sheet를 DataFrame 객체로 읽어 들인다.
df = pd.read_excel('https://github.com/joongyang/Machine-Learning-by-Examples/blob/master/Real%20Estate%20Price.xlsx?raw=true')
df.head()

,No,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price of unit area
0,1,2012.916667,32.0,84.87882,10,24.98298,121.54024,37.9
1,2,2012.916667,19.5,306.59470,9,24.98034,121.53951,42.2
2,3,2013.583333,13.3,561.98450,5,24.98746,121.54391,47.3
3,4,2013.500000,13.3,561.98450,5,24.98746,121.54391,54.8
4,5,2012.833333,5.0,390.56840,5,24.97937,121.54245,43.1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414 entries, 0 to 413
Data columns (total 8 columns):
No                                        414 non-null int64
X1 transaction date                       414 non-null float64
X2 house age                              414 non-null float64
X3 distance to the nearest MRT station    414 non-null float64
X4 number of convenience stores           414 non-null int64
X5 latitude                               414 non-null float64
X6 longitude                              414 non-null float64
Y house price of unit area                414 non-null float64
dtypes: float64(6), int64(2)
memory usage: 26.0 KB


## Data 설명
---
[주택 가격 자료](https://archive.ics.uci.edu/ml/datasets/Real+estate+valuation+data+set)

대만 New Taipei시 Sindian 지역에서 수집한 주택 가격 자료이다.

## 변수 설명
---

|칼럼명 | 설명|
|:---|:---|
| No | 일련 번호 |
| X1 transaction date | 거래 날짜(예: 2013.250 = 2013 March, 2013.500 = 2013 June, etc.) |
| X2 house age | 주택 연령(단위: 연) |
| X3 distance to the nearest MRT station | 가장 가까운 지하철 역까지의 거리(단위: 미터) |
| X4 number of convenience stores | 생활권 내에 위치한 편의점 수 |
| X5 latitude | 위도(부동산의 위치) |
| X6 longitude | 경도(부동산의 위치) |
| Y house price of unit area | 평당 부동산 가격(단위: 100,000 대만 달러) |


In [5]:
df.columns = ['No', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'Y']
df.columns

Index(['No', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'Y'], dtype='object')

In [6]:
df.head()

,No,X1,X2,X3,X4,X5,X6,Y
0,1,2012.916667,32.0,84.87882,10,24.98298,121.54024,37.9
1,2,2012.916667,19.5,306.59470,9,24.98034,121.53951,42.2
2,3,2013.583333,13.3,561.98450,5,24.98746,121.54391,47.3
3,4,2013.500000,13.3,561.98450,5,24.98746,121.54391,54.8
4,5,2012.833333,5.0,390.56840,5,24.97937,121.54245,43.1


## sklearn api documentation 검색 결과의 이해

## 자료의 분할

In [0]:
Y = df['Y']
X = df[['X2', 'X3', 'X4', 'X5', 'X6']]

In [0]:
from sklearn.model_selection import train_test_split
X_tr, X_ts, Y_tr, Y_ts = train_test_split(X, Y,
                                        test_size=0.3, random_state=20191028)

In [9]:
print(X_tr.shape, X_ts.shape, Y_tr.shape, Y_ts.shape)

(289, 5) (125, 5) (289,) (125,)


## Linear Regression

In [10]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()

reg.fit(X_tr, Y_tr)
Y_tr_pred = reg.predict(X_tr)

print(reg.score(X_tr, Y_tr))

from sklearn.metrics import mean_squared_error
mse_tr = mean_squared_error(Y_tr, Y_tr_pred)

print(mse_tr)


0.6231482040797469
66.50281456803764


In [11]:
Y_ts_pred = reg.predict(X_ts)
mse_ts = mean_squared_error(Y_ts, Y_ts_pred)
print(mse_ts)
print(reg.score(X_ts, Y_ts))

109.6653662108158
0.4609533885095439


## Ridge Regression과 cross-validation

In [12]:
from sklearn.linear_model import Ridge
reg = Ridge(alpha=1.0)

reg.fit(X_tr, Y_tr)
Y_tr_pred = reg.predict(X_tr)
# R square
print(reg.score(X_tr, Y_tr)) 
# MSE
mse_tr = mean_squared_error(Y_tr, Y_tr_pred)
print(mse_tr)

0.5964170526169272
71.22004512967257


#### 훈련 자료로 훈련된 모형의 예측 능력은 테스트 자료로 평가한다.

In [13]:
# 테스트 자료에 대한 R^2
print(reg.score(X_ts, Y_ts))
# 테스트 자료에 대한 MSE
Y_ts_pred = reg.predict(X_ts)
mse_ts = mean_squared_error(Y_ts, Y_ts_pred)
print(mse_ts)

0.4327191566512436
115.40942861733073


#### Ridge regression 모형의 hyperparameter인 $\alpha$의 최적값을 선택하려면 cross-validation을 이용한다.

sklearn.linear_model 모듈에 이를 위한 RidgeCV 클래스가 준비되어 있다.
RidgeCV는 지정된 여러 $\alpha$ 중에서 최적의 것을 CV로 찾아주는 일만 한다.

In [14]:
from sklearn.linear_model import RidgeCV
alpha_vals = [0.001, 0.01, 0.05, 0.1, 0.2, 0.5, 1, 2, 3, 5]
reg = RidgeCV(alphas=alpha_vals)

reg.fit(X_tr, Y_tr)
print(reg.alpha_)

0.001


In [15]:
import numpy as np
alpha_vals = np.arange(0.0005, 0.005, 0.0005)
reg = RidgeCV(alphas=alpha_vals)

reg.fit(X_tr, Y_tr)
print(reg.alpha_)

0.002


#### 최적 $\alpha$ 값을 사용한 Ridge regression 모형은 모든 훈련 자료를 사용하여 훈련하여 그 예측력을 테스트 자료로 평가할 수 있다.

In [16]:
reg = Ridge(alpha=0.002)

reg.fit(X_tr, Y_tr)
print(reg.score(X_tr, Y_tr))

0.6229472022637852


In [17]:
Y_tr_pred = reg.predict(X_tr)
mse_tr = mean_squared_error(Y_tr, Y_tr_pred)
print(mse_tr)

66.53828524016778


In [18]:
print(reg.score(X_ts, Y_ts))

Y_ts_pred = reg.predict(X_ts)
mse_ts = mean_squared_error(Y_ts, Y_ts_pred)
print(mse_ts)

0.4622755857991425
109.39637416657337


## Lasso 모형에 대한 cross-validation

In [28]:
from sklearn.linear_model import LassoCV
alphas = np.arange(0.1, 3, 0.2)
lasso = LassoCV(alphas=alphas)

lasso.fit(X_tr, Y_tr)
lasso.alpha_

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


0.1

In [29]:
alphas = np.arange(0.002, 0.2, 0.002)
lasso = LassoCV(alphas=alphas)

lasso.fit(X_tr, Y_tr)
lasso.alpha_

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


0.004

In [30]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.004)
lasso.fit(X_tr, Y_tr)
print(lasso.score(X_tr, Y_tr))
Y_tr_pred = lasso.predict(X_tr)
print(mean_squared_error(Y_tr, Y_tr_pred))

0.6213375743425822
66.82233533181162


In [31]:
print(lasso.score(X_ts, Y_ts))
Y_ts_pred = lasso.predict(X_ts)
print(mean_squared_error(Y_ts, Y_ts_pred))

0.4652340280393368
108.7944992177867


## Ridge regression과 Lasso를 비교하는 위의 방법은 엄밀히 옳지 않다.

만약 목적이 Ridge regression model과 Lasso model 중에서 최적의 모형을 찾는 것이 목적이라면 위의 방법은 잘못되었다.
왜냐하면 최적의 ridge regression model과 최적의 lasso model을 비교하기 위해 테스트 자료가 사용되었기 때문이다.
테스트 자료는 최종적으로 선택된 모형의 예측 성능을 평가할 때만 사용되어야 한다.
최적의 ridge regression model과 최적의 lasso model을 비교할 때는 CV에서 얻어진 예측성능을 비교하여 결정하여야 한다.

이를 위해서는 GridSearchCV를 사용하는 것이 편리하다.
일반적으로 RidgeCV, LassoCV와 같이 특별한 모형을 위한 CV보다 GridSearchCV를 사용하는 것이 바람직하다.

In [32]:
from sklearn.model_selection import GridSearchCV
ridge = Ridge()
alpha_vals = [0.0001, 0.001, 0.01, 0.1, 1, 2, 3, 5]
ridge_params = {'alpha': alpha_vals}
ridge_gs = GridSearchCV(ridge, param_grid = ridge_params,
                        cv=10, refit=True)
ridge_gs.fit(X_tr, Y_tr)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='warn', n_jobs=None,
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 2, 3, 5]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [33]:
ridge_gs.best_estimator_

Ridge(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

## 1차로 찾은 최적값 주변에서 다시 CV를 실행해본다.

In [34]:
alpha_vals = [0.0002, 0.005, 0.002]
ridge_params = {'alpha': alpha_vals}
ridge_gs = GridSearchCV(ridge, param_grid = ridge_params,
                        cv=10, refit=True)
ridge_gs.fit(X_tr, Y_tr)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='warn', n_jobs=None,
             param_grid={'alpha': [0.0002, 0.005, 0.002]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [35]:
ridge_gs.best_params_

{'alpha': 0.005}

#### 최적 $\alpha$에 대한 CV를 실행한 결과

매개변수 cv에 10을 주었기 때문에 교차검증에 의해 10개의 예측력 평가 점수가 나오는데 이 10개의 평균을 최적 $\alpha$의 CV 결과로 사용한다.

In [36]:
print(ridge_gs.best_score_)

0.5760126554078486


GridSearchCV의 refit 매개변수에 True를 주면, fit() 메소드를 실행한 결과는 CV로 찾은 최적 $\alpha$ 값을 이용한 Ridge regression 모형을 전체 훈련 자료로 훈련한 모형이다.
따라서 predict() 메소드로 바로 예측할 수 있다.

In [37]:
Y_tr_pred = ridge_gs.predict(X_tr)
mean_squared_error(Y_tr, Y_tr_pred)

66.67586674094515

In [39]:
Y_ts_pred = ridge_gs.predict(X_ts)
mean_squared_error(Y_ts, Y_ts_pred)

109.26969830259401

#### GridSearchCV를 이용하여 Lasso에 대한 CV를 실시하고 최적의 Lasso 모형과 최적의 Ridge 모형을 비교하시오.

In [41]:
lasso = Lasso()
lasso_alphas = np.arange(0.001, 3, 0.1)
lasso_params = {'alpha': lasso_alphas}
lasso_gs = GridSearchCV(lasso, param_grid = lasso_params, cv=10)
lasso_gs.fit(X_tr, Y_tr)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=1000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'alpha': array([1.000e-03, 1.010e-01, 2.010e-01, 3.010e-01, 4.010e-01, 5.010e-01,
       6.010e-01, 7.010e-01, 8.010e-01, 9.010e-01, 1.001e+00, 1.101e+00,
       1.201e+00, 1.301e+00, 1.401e+00, 1.501e+00, 1.601e+00, 1.701e+00,
       1.801e+00, 1.901e+00, 2.001e+00, 2.101e+00, 2.201e+00, 2.301e+00,
       2.401e+00, 2.501e+00, 2.601e+00, 2.701e+00, 2.801e+00, 2.901e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [42]:
lasso_gs.best_score_

0.5755800478886928

In [43]:
print(lasso_gs.score(X_ts, Y_ts))
Y_ts_pred = lasso_gs.predict(X_ts)
mean_squared_error(Y_ts, Y_ts_pred)

0.46333797976613145


109.18023734098819

#### Ridge 모형과 Lasso 모형 중에서 선택된 최적 모형의 성능을 테스트 자료로 평가해보시오.

## Kernel trick을 이용한 ridge regression

In [52]:
from sklearn.kernel_ridge import KernelRidge
kr = KernelRidge()
kr_params = {'alpha': np.arange(0.01, 3, 0.2), 
             'kernel': ['rbf'] }
kr_gs = GridSearchCV(kr, param_grid=kr_params, cv=10)

kr_gs.fit(X_tr, Y_tr)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=KernelRidge(alpha=1, coef0=1, degree=3, gamma=None,
                                   kernel='linear', kernel_params=None),
             iid='warn', n_jobs=None,
             param_grid={'alpha': array([0.01, 0.21, 0.41, 0.61, 0.81, 1.01, 1.21, 1.41, 1.61, 1.81, 2.01,
       2.21, 2.41, 2.61, 2.81]),
                         'kernel': ['rbf']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [53]:
kr_gs.best_params_

{'alpha': 0.01, 'kernel': 'rbf'}

In [54]:
kr_gs.best_score_  # 0 <= R^2 <= 1, 클수록 좋다.

-4.303952822695925

In [55]:
kr_gs.best_estimator_

KernelRidge(alpha=0.01, coef0=1, degree=3, gamma=None, kernel='rbf',
            kernel_params=None)

In [50]:
Y_tr_pred = kr_gs.predict(X_tr)
mean_squared_error(Y_tr, Y_tr_pred)

2.154220337576654

In [51]:
Y_ts_pred = kr_gs.predict(X_ts)
mean_squared_error(Y_ts, Y_ts_pred)

666.0866737407224

## Support vector regression과 cross-validation

In [64]:
from sklearn.svm import SVR
svr = SVR()
svr_param = {'kernel': ['rbf', 'sigmoid'],
             'C': np.arange(0.2, 5, 0.1),
             'degree': [2, 3]
}
svr_gs = GridSearchCV(svr, param_grid=svr_param, cv=10)
svr_gs.fit(X_tr, Y_tr)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default 

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='auto_deprecated', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': array([0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3, 1.4,
       1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7,
       2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4. ,
       4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9]),
                         'degree': [2, 3], 'kernel': ['rbf', 'sigmoid']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [65]:
svr_gs.best_estimator_

SVR(C=4.900000000000002, cache_size=200, coef0=0.0, degree=2, epsilon=0.1,
    gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)

In [66]:
svr_gs.best_score_

0.37632980592615334

## Cross-Validation을 위해 Fold로 나누는 방법 제어하기

## 성능을 높이기 위한 정규화와 표준화의 이용

## 기계 학습의 일반적인 절차

## Real Estate Price Data에 대해 더 좋은 모형을 얻으려면...